
Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd

In [58]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [59]:
ratings['date'] = pd.to_datetime(ratings['timestamp'], unit='s')
ratings.head()

,userId,movieId,rating,timestamp,date
0,1,31,2.5,1260759144,2009-12-14 02:52:24
1,1,1029,3.0,1260759179,2009-12-14 02:52:59
2,1,1061,3.0,1260759182,2009-12-14 02:53:02
3,1,1129,2.0,1260759185,2009-12-14 02:53:05
4,1,1172,4.0,1260759205,2009-12-14 02:53:25


In [60]:
user_info = ratings.groupby('userId').agg({'date': ['min', 'max'], 'rating' : 'count'}).reset_index()

In [62]:
user_info.head()

userId                date                    
                          min                 max
3       4 2000-02-05 19:25:14 2000-02-08 03:57:54
7       8 2006-07-31 23:42:20 2006-08-01 23:22:07
14     15 2001-08-16 04:47:19 2016-07-24 03:25:35
16     17 2005-09-23 09:43:07 2005-09-23 11:57:20
18     19 1997-02-06 00:48:11 1997-02-06 02:16:13

In [61]:
user_info = user_info[user_info['rating']['count'] > 100][['userId', 'date']]

In [64]:
user_info['dif'] = user_info['date']['max'] - user_info['date']['min']

In [67]:
user_info.dif.mean()

Timedelta('463 days 21:28:27.449612')

Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

таблицу с тремя типами выручки для каждого client_id без указания адреса клиента

аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

In [2]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [3]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [4]:

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [5]:

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [10]:
client_adress = client_base.merge(rzd, how='outer', on = 'client_id').merge(auto, how='outer', on = 'client_id').merge(air,how='outer', on = 'client_id')
client_logs = rzd.merge(auto, how='outer', on = 'client_id').merge(air,how='outer', on = 'client_id')

In [12]:
client_adress.head(20)

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Допустим, нам известны пары координат (широта, долгота) для каждого пользователя и они упорядоченно добавлены в список, данные сохранены например в виде словаря (ключ - user_id, значение - список с координатами). 
Тогда в лог с визитами и покупками  можно добавить пары координат по порядку увеличения индексов в списке координат. 
Мы сможем посмотреть частоту появления одинаковых или похожих координат в списке для каждого пользователя и определить удобные расположения и сделать ему предложения, актуальные для этого адреса.
Также по разнице координат для одного пользователя можно понять, как часто и на какие расстояния он перемещается. Если связать с покупками, то можно узнать как он обычно покупает товары, купил ли он товары после визита и какие именно товары интересовали его определенном месте.